# Braille preliminaries
It is written in 2 columns of 3 rows.

The dots are named 1 2 3 top to bottom on the left, 4 5 6 top to bottom on the right.

I name them so that 123456 corresponds to BRAILE.

I do the follwing encodings:
- EACH: Summing all of the same dots in a word. So "hi" which is 125 24 becomes (1,2,0,1,1,0)
- BRA: a word becomes (l,r) where l is the sum of the left dots (123) and r is the sum of the right dots (456)
- BAIE: a word becomes (c,e) where c is the sum of all dots in the corners (1346) and e is the sum of all dots in the inn (25)
- BI: a word becomes (t,c,b) where t is top (14), c is center (25), b is bottom (36)

Then the decoding is ambiguous, and that is most of the fun.

In [40]:
import matplotlib.pyplot as plt
import random
import ast

# Read info from initial format
def get_char_dots():
    char_dots={}
    with open("braille.txt", "r") as f:
        for line in f.readlines()[1:]:
            sp=line.split("\t")
            char_dots[sp[0]]=sp[2].rstrip()
    #print(char_dots)
    return char_dots

# Aggregates dict values and turns them into keys
# So {"a":1,"b":2,"c":1} becomes {1:["a","c"],2:["b"]}
def aggregate_dictionary(di):
    ag_di={}
    for k,v in di.items():
        a=ag_di.get(v,[])
        a.append(k)
        ag_di[v]=a
    return ag_di

char_dots=get_char_dots()

In [10]:
# Cell specific to BRA (or leri)

# Helper function to separate left from right
left=set(["1","2","3"])
right=set(["4","5","6"])
def get_leri_tup(st):
    le=0
    ri=0
    for char in st:
        if char in left:
            le+=1
        elif char in right:
            ri+=1
    return (le,ri)

#In this dict, first element of value is left, second is right
char_leri_tups={k:get_leri_tup(v) for k,v in char_dots.items()}

# Converting text to ambiguous code
def sum_leri(word):
    le=0
    ri=0
    for ch in word:
        tup=char_leri_tups.get(ch,(-1000,-1000))
        le+=tup[0]
        ri+=tup[1]
    return (le,ri)

def text_to_leri(text):
    text=text.lower().replace("."," .").replace(","," ,").replace(":"," :").replace(";"," ;").replace("?"," ?").replace("!"," !").replace("'"," '")
    converted=[]
    for w in text.split():
        leri=sum_leri(w)
        if leri[0]>=0:
            converted.append(leri)
    return converted


# Now preparing for the reverse, so that all pairs map to some words
def get_words_leri():
    words_leri={}
    with open("words_en.txt","r") as f:
        for line in f.readlines():
            w=line.split()[0]
            su=sum_leri(w)
            if su[0]>=0:
                words_leri[w]=su

    return words_leri


def plot_leri(words_leri):
    le=[d[0]+random.random()/5 for d in words_leri.values()]
    ri=[d[1]+random.random()/5 for d in words_leri.values()]
    plt.scatter(le,ri,s=1)
    plt.show()

words_leri=get_words_leri()
leri_words=aggregate_dictionary(words_leri)

# It works like so
print(words_leri["irene"])

text="Hi my name is Irene and I like flowers"
leri_list=text_to_leri(text)
print(leri_list)
for leri in leri_list:
    print(leri_words[leri][:15])


(8, 6)
[(3, 2), (4, 4), (6, 4), (3, 2), (8, 6), (4, 4), (1, 1), (7, 2), (14, 8)]
["'t", 'is', 'me', 'he', 'if', 'at', 'an', 'hi', 'em', 'eh', "'ii", 'na', 'cia', 'oi', 'aii']
['and', 'my', 'was', 'saw', 'idea', 'joe', 'yo', 'jim', 'due', 'doc', 'aye', 'tie', 'dan', 'dna', 'fed']
['out', 'come', 'been', 'too', 'mean', 'mr.', 'name', 'son', 'boy', 'kids', 'pay', 'dear', 'fun', 'game', 'read']
["'t", 'is', 'me', 'he', 'if', 'at', 'an', 'hi', 'em', 'eh', "'ii", 'na', 'cia', 'oi', 'aii']
['these', 'being', 'while', 'times', 'hands', 'chance', 'sense', 'jesus', 'song', 'state', 'choice', 'meant', 'swear', 'ring', 'quit']
['and', 'my', 'was', 'saw', 'idea', 'joe', 'yo', 'jim', 'due', 'doc', 'aye', 'tie', 'dan', 'dna', 'fed']
[':', '-', 'i', 'c', 'e']
['like', 'calm', 'bank', 'bear', 'pack', 'bro', 'male', 'alan', 'lisa', 'meal', 'mail', 'rob', 'fail', 'sale', 'sara']
['business', 'telling', 'shouldn', 'parents', 'speaking', 'forgive', 'quickly', 'helping', 'flowers', 'language', 'national', '

In [11]:
# Cell specific to BAIE (or corins)

# Helper function to separate centers from corners
# Why this distinction? If you were poking a paper with a machine that makes circles,
# you'd get quarter circles for the corners and semicircles for the ins.
# After drilling a word you only know how many of each you have.
corners=set(["1","3","4","6"])
ins=set(["2","5"])
def get_corins_tup(st):
    cor=0
    inn=0
    for char in st:
        if char in corners:
            cor+=1
        elif char in ins:
            inn+=1
    return (cor,inn)

#In this dict, first element of value is corners, second is ins
char_corins_tups={k:get_corins_tup(v) for k,v in char_dots.items()}

# Converting text to ambiguous code
def sum_corins(word):
    cor=0
    inn=0
    for ch in word:
        tup=char_corins_tups.get(ch,(-1000,-1000))
        cor+=tup[0]
        inn+=tup[1]
    return (cor,inn)

def text_to_corin(text):
    text=text.lower().replace("."," .").replace(","," ,").replace(":"," :").replace(";"," ;").replace("?"," ?").replace("!"," !").replace("'"," '")
    converted=[]
    for w in text.split():
        corins=sum_corins(w)
        if corins[0]>=0:
            converted.append(corins)
    return converted


# Now preparing for the reverse, so that all pairs map to some words
def get_words_corins():
    words_corins={}
    with open("words_en.txt","r") as f:
        for line in f.readlines():
            w=line.split()[0]
            su=sum_corins(w)
            if su[0]>=0:
                words_corins[w]=su

    return words_corins


def plot_corins(corins):
    cor=[d[0]+random.random()/5 for d in corins.values()]
    edg=[d[1]+random.random()/5 for d in corins.values()]
    plt.scatter(cor,edg,s=1)
    plt.show()

words_corins=get_words_corins()
corin_words=aggregate_dictionary(words_corins)
#plot_corins(words_corins)

# It works like so
print(words_corins["irene"])

text="Hi my name is Irene and I like flowers"
corins_list=text_to_corin(text)
print(corins_list)
for corin in corins_list:
    print(corin_words[corin][:15])


(8, 6)
[(2, 3), (7, 1), (8, 2), (3, 2), (8, 6), (6, 2), (1, 1), (6, 3), (13, 9)]
['he', 'hi', 'eh', 'i.', 'b.', 'ji', 'e.', 'je', 'bj', 'jb', '.i', 'ej', 'ij', 'ih', 'hb']
['my', 'man', 'make', 'came', 'kick', 'adam', 'map', 'lack', 'bum', 'sack', 'pam', 'yu', 'kay', 'cuba', 'nam']
['come', 'mean', 'any', 'name', 'pay', 'fun', 'buy', 'black', 'pick', 'sun', 'club', 'mark', 'nick', 'alex', 'lock']
["'t", 'is', 'if', 'at', 'aah', 'de', "'ii", 'aw', 'ed', 'id', 'el', 'le', 'oi', 'li', 'bo']
['they', 'after', 'hello', 'being', 'went', 'later', 'watch', 'word', 'hurt', 'sort', 'state', 'swear', 'ring', 'idiot', 'trip']
['and', 'use', 'face', 'case', 'jack', 'sake', 'bus', 'yo', 'arm', 'due', 'doc', 'fake', "'all", 'aye', 'dan']
[';', 'i', 'e', 'b']
['but', 'like', 'one', 'take', 'said', 'off', 'old', 'isn', 'care', 'dead', 'end', 'deal', 'met', 'bye', 'safe']
['telling', 'strong', 'wedding', 'strange', 'forgive', 'forever', 'finished', 'forward', 'holding', 'helping', 'witness', 'flowers', 

In [12]:
# Cell specific to BI (or tomibo)

# Helper function to separate top middle bottom
top=set(["1","4"])
mid=set(["2","5"])
bot=set(["3","6"])

def get_tomibo_tup(st):
    to=0
    mi=0
    bo=0
    for char in st:
        if char in top:
            to+=1
        elif char in mid:
            mi+=1
        elif char in bot:
            bo+=1
    return (to,mi,bo)

#In this dict, first element of value is top, second is middle, last is right
char_tomibo_tups={k:get_tomibo_tup(v) for k,v in char_dots.items()}

# Converting text to ambiguous code
def sum_tomibo(word):
    to=0
    mi=0
    bo=0
    for ch in word:
        tup=char_tomibo_tups.get(ch,(-1000,-1000,-1000))
        to+=tup[0]
        mi+=tup[1]
        bo+=tup[2]
    return (to,mi,bo)

def text_to_tomibo(text):
    text=text.lower().replace("."," .").replace(","," ,").replace(":"," :").replace(";"," ;").replace("?"," ?").replace("!"," !").replace("'"," '")
    converted=[]
    for w in text.split():
        tomibo=sum_tomibo(w)
        if tomibo[0]>=0:
            converted.append(tomibo)
    return converted


# Now preparing for the reverse, so that all pairs map to some words
def get_words_tomibo():
    words_tomibo={}
    with open("words_en.txt","r") as f:
        for line in f.readlines():
            w=line.split()[0]
            su=sum_tomibo(w)
            if su[0]>=0:
                words_tomibo[w]=su

    return words_tomibo


words_tomibo=get_words_tomibo()
tomibo_words=aggregate_dictionary(words_tomibo)

# It works like so
print(words_tomibo["irene"])

text="Hi my name is Irene and I like flowers"
tomibo_list=text_to_tomibo(text)
print(tomibo_list)
for tomibo in tomibo_list:
    print(tomibo_words[tomibo][:15])


(6, 6, 2)
[(2, 3, 0), (4, 1, 3), (6, 2, 2), (2, 2, 1), (6, 6, 2), (5, 2, 1), (1, 1, 0), (4, 3, 2), (8, 9, 5)]
['he', 'hi', 'eh', 'ji', 'je', 'bj', 'jb', 'ej', 'ij', 'ih', 'hb', 'bh']
['my', 'bum', 'kay', 'axl', 'luc', 'anu', 'una', 'yak', 'lax', 'csu', 'sax', 'apu', 'mui', 'usc', 'kms']
['come', 'mean', 'name', 'black', 'pick', 'nick', 'neck', 'main', 'papa', 'anna', 'amen', 'clan', 'scan', 'backs', 'dock']
['is', 'at', "'ii", 'aw', 'el', 'le', 'oi', 'li', 'bo', 'si', 'se', 'ta', 'wa', 'es', 'c.']
['after', 'watch', 'ring', 'idiot', 'stage', 'twice', 'large', 'chris', 'easier', 'birds', 'chest', 'trade', 'theme', 'helen', 'bored']
['and', 'case', 'jack', 'doc', 'fake', 'dan', 'fan', 'dna', 'beam', 'hack', 'pad', 'alec', 'ciao', 'med', 'beck']
['i', 'e', 'b']
['like', 'one', 'take', 'old', 'isn', 'met', 'bye', 'eye', 'wake', 'ran', 'ball', 'weak', 'kate', 'tim', 'lisa']
['strong', 'forever', 'witness', 'flowers', 'towards', 'airport', 'warrant', 'robbery', 'spirits', 'liberty', 'mothers

# Story
So we have Leri, Corin and Tomibo. Each of them has a braille typewriter that is very special in the way it works.

For Leri, each time a key is pressed, it counts separately the number of dots on the left and the number of dots on the right. Those two values accumulate by summing on each keypress, and when space is entered, the only thing that remains from a word is a tuple (le,ri) where le are the total dots on the left and ri are the total dots on the right.

For Corin it is similar. But what is count separately is the number of dots on the corners or the inner ones.

Similar for Tomibo, now top, middle and bottom dots are counted separately.

They communicate only through their typewriters, but as the narrator I will indicate who is speaking.

In [43]:
def encode_leri(text):
    return "Leri    "+str(text_to_leri(text))

def encode_corin(text):
    return "Corin   "+str(text_to_corin(text))

def encode_tomibo(text):
    return "Tomibo  "+str(text_to_tomibo(text))


In [62]:
def decode_leri(leri_list,options=15):
    for i,leri in enumerate(leri_list):
        print(i,leri,leri_words[leri][:options])

def decode_corin(corin_list,options=15):
    for i,corin in enumerate(corin_list):
        print(i,corin,corin_words[corin][:options])
        
def decode_tomibo(tomibo_list,options=15):
    for i,tomibo in enumerate(tomibo_list):
        print(i,tomibo,tomibo_words[tomibo][:options])

def str_to_list(st):
    return ast.literal_eval(st)
    
def decode(line,options=30):
    content=str_to_list(line[8:])
    match line[:8]:
        case "Leri    ":
            decode_leri(content,options)
        case "Corin   ":
            decode_corin(content,options)
        case "Tomibo  ":
            decode_tomibo(content,options)

In [102]:
# Example
text="Nice to meet you, I am a bit ambiguous."
print("Text\n",text)

print("\nEncoding:")
l1=encode_leri(text)
print(l1)

print("\nDecoding")
decode(l1)

print("\n",leri_words[(16,8)])

Text
 Nice to meet you, I am a bit ambiguous.

Encoding:
Leri    [(5, 5), (4, 3), (6, 5), (6, 5), (1, 0), (1, 1), (3, 1), (1, 0), (5, 3), (16, 8), (1, 2)]

Decoding
0 (5, 5) ['don', 'get', 'how', 'who', 'yes', 'hey', 'any', 'god', 'nice', 'week', 'side', 'dr.', 'dog', 'ten', 'date', 'hide', 'gay', 'jane', 'code', 'feed', 'dies', 'dean', 'egg', 'st.', 'jean', 'a.m.', 'woo', 'no.', 'net', 'gig']
1 (4, 3) ['to', 'on', 'no', 'can', 'go', 'see', 'had', 'by', 'eat', 'kid', 'ain', 'bed', 'mad', 'act', 'age', 'tea', 'sad', 'cat', 'dr', 'ate', 'sec', 'ian', 'st', 'jam', 'bid', 'fee', "'t-", 'un', 'chi', 'tae']
2 (6, 5) ['you', 'not', 'yeah', 'got', 'time', 'does', 'fine', 'kind', 'seen', 'once', 'used', 'meet', 'hand', 'damn', 'easy', 'mine', 'whoa', 'gun', 'feet', 'ride', 'wear', 'deep', 'cry', 'cute', 'yep', 'hadn', 'size', 'ideas', 'jeff', 'wash']
3 (6, 5) ['you', 'not', 'yeah', 'got', 'time', 'does', 'fine', 'kind', 'seen', 'once', 'used', 'meet', 'hand', 'damn', 'easy', 'mine', 'whoa', 'gu

# Preliminaries

From less to more ambiguous: Tomibo < Corin < Leri

Some care was put so that it is possible to decipher, but as for the difficulty, it is what it is.

# And the story goes something like this

Leri, Corin and Tomibo are roommates, and they have a pet turtle. Well had, because it just died.

They want to figure out what happened to the turtle, and you can figure out what happened by decoding what they say.



In [ ]:
"""
Tomibo  [(1, 1, 0), (3, 2, 1), (4, 4, 3), (7, 7, 3), (2, 3, 1), (3, 3, 2), (8, 6, 6), (0, 1, 0), (11, 9, 6), (3, 3, 2), (8, 4, 4), (6, 3, 0), (3, 3, 4), (3, 2, 1), (3, 4, 2), (10, 5, 7), (0, 2, 1)]
Leri    [(6, 3), (9, 6), (2, 4), (7, 3), (7, 5), (4, 3), (7, 5), (9, 3), (1, 2)]
Corin   [(1, 1), (10, 5), (17, 9), (1, 2)]
Leri    [(4, 2), (6, 5), (11, 4), (3, 3), (1, 0), (6, 5), (5, 2), (1, 0), (7, 5), (4, 2), (7, 5), (1, 2)]
Corin   [(5, 2), (0, 1), (1, 1), (5, 3), (7, 4), (1, 2)]
Tomibo  [(3, 4, 1), (5, 3, 5), (4, 4, 2), (5, 3, 1), (12, 11, 5), (4, 4, 2), (0, 2, 1), (3, 4, 2), (1, 1, 2), (11, 7, 6), (0, 2, 1)]
Leri    [(5, 5), (4, 4), (5, 4), (8, 3), (5, 4), (4, 3), (4, 3), (17, 9), (9, 3), (2, 1)]
Tomibo  [(1, 1, 0), (3, 3, 2), (2, 3, 1), (12, 10, 7), (11, 8, 5), (0, 1, 0), (2, 2, 1), (11, 8, 1), (5, 4, 2), (0, 2, 1)]
Leri    [(1, 1), (5, 4), (3, 3), (15, 14), (12, 10), (1, 0), (9, 5), (8, 5), (1, 2)]
Corin   [(1, 1), (4, 2), (7, 4), (14, 8), (0, 1), (10, 3), (6, 5), (9, 3), (5, 3), (1, 2)]
Tomibo  [(4, 6, 2), (8, 5, 6), (12, 7, 9), (0, 1, 0), (4, 6, 2), (4, 6, 2), (4, 2, 5), (8, 6, 2), (0, 1, 2)]
Corin   [(8, 5), (14, 7), (5, 2), (0, 1), (1, 1), (5, 3), (7, 4), (4, 3), (17, 10), (13, 9), (1, 2)]
Leri    [(4, 2), (1, 1), (4, 4), (5, 4), (8, 3), (5, 4), (4, 3), (4, 3), (17, 9), (9, 3), (1, 2)]
Tomibo  [(5, 3, 0), (4, 2, 5), (8, 6, 3), (12, 10, 5), (6, 7, 2), (0, 1, 2)]
Leri    [(15, 7), (5, 5), (1, 0), (3, 3), (4, 4), (13, 9), (7, 3), (5, 2), (4, 2), (1, 1), (10, 9), (5, 4), (8, 7), (1, 2)]
Tomibo  [(2, 1, 2), (0, 1, 0), (3, 4, 2), (1, 1, 2), (3, 3, 1), (3, 5, 1), (5, 7, 3), (0, 2, 1)]
Corin   [(3, 3), (7, 4), (7, 4), (10, 4), (6, 3), (17, 10), (1, 2)]
Tomibo  [(1, 1, 0), (3, 2, 1), (4, 4, 3), (8, 6, 3), (5, 2, 3), (6, 4, 4), (6, 9, 2), (0, 1, 0), (5, 2, 1), (3, 5, 1), (5, 7, 3), (5, 4, 5), (6, 4, 1), (0, 2, 1)]
Corin   [(6, 5), (0, 1), (5, 3), (9, 2), (14, 8), (4, 3), (8, 3), (7, 4), (4, 2), (4, 5), (8, 7), (2, 1)]
Leri    [(6, 5), (1, 0), (1, 1), (7, 4), (9, 4), (4, 2), (8, 3), (1, 0), (5, 7), (2, 1)]
Tomibo  [(5, 5, 4), (4, 6, 2), (0, 1, 0), (4, 6, 2), (2, 2, 1), (3, 4, 3), (6, 2, 3), (4, 4, 3), (0, 1, 0), (4, 2, 5), (7, 5, 3), (2, 3, 1), (0, 2, 1)]
"""